<a href="https://colab.research.google.com/github/nannthd/Project/blob/main/yolo07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ติดตั้ง YOLOv5

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -U -r requirements.txt

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'})")

In [45]:
from pathlib import Path
from PIL import Image

#สำหรับการแสดงผล
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

#Download Custom Dataset

In [46]:
# เชื่อมdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
from torchvision import datasets, transforms

# ระบุที่อยู่ของ Dataset ใน Google Drive ของคุณ
dataset_path = '/content/drive/MyDrive/Chromosome dataset'

# ใช้ torchvision เพื่อโหลด Dataset
dataset = datasets.ImageFolder(
    dataset_path,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
)

In [48]:
import matplotlib.pyplot as plt
import numpy as np

# จำนวนข้อมูลใน Dataset
num_samples = len(dataset)
print(f"Number of samples in the dataset: {num_samples}")

Number of samples in the dataset: 72


In [49]:
# ใช้ !cat เพื่อแสดงเนื้อหาของไฟล์ data.yaml
!cat '/content/drive/MyDrive/Chromosome dataset/data.yaml'

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['chromosome']

roboflow:
  workspace: kku-o5mso
  project: chromosome-v.3
  version: 8
  license: CC BY 4.0
  url: https://universe.roboflow.com/kku-o5mso/chromosome-v.3/dataset/8

# Define Model Configuration and Architecture

In [50]:
# ระบุที่อยู่ของ Dataset ใน Google Drive ของคุณ
dataset_path = '/content/drive/MyDrive/Chromosome dataset'

# เปิดไฟล์ data.yaml โดยใช้ dataset_path
with open(dataset_path + "/data.yaml", 'r') as stream:
    # อ่านข้อมูลจากไฟล์ data.yaml
    num_classes = str(yaml.safe_load(stream)['nc'])

# แสดงจำนวนคลาส
print(f"Number of classes: {num_classes}")

Number of classes: 1


In [51]:
#this is the model configuration we will use for our tutorial
%cat /content/yolov5/models/yolov5m.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.67 # model depth multiple
width_multiple: 0.75 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 

In [53]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [54]:
%%writetemplate /content/yolov5/models/custom_yolov5m.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train

In [55]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 200 --data /content/drive/MyDrive/Chromosome\ dataset/data.yaml --cfg ./models/custom_yolov5m.yaml --weights 'yolov5m.pt' --optimizer SGD --freeze 10 --image-weights --hyp ./data/hyps/hyp.scratch-high.yaml --name yolov5m_results  --cache

/content/yolov5
2024-01-25 07:33:19.090980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 07:33:19.091056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 07:33:19.094083: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=./models/custom_yolov5m.yaml, data=/content/drive/MyDrive/Chromosome dataset/data.yaml, hyp=./data/hyps/hyp.scratch-high.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, buc

In [6]:
# ทำนาย Object Detection
results = model(img)

In [8]:
# แสดงผลลัพธ์
results.show()

In [9]:
# แสดงผลลัพธ์ด้วย matplotlib (ต้องการโค้ดเพิ่มเติมสำหรับการแสดงผล)
def plot_results(img, results):
    plt.imshow(img)
    ax = plt.gca()

    for det in results.xyxy[0].numpy():
        rect = Rectangle((det[0], det[1]), det[2] - det[0], det[3] - det[1], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()

plot_results(img, results)

In [10]:
try:
    results = model(img)
    results.show()
    plot_results(img, results)
except Exception as e:
    print(f"Error: {e}")